# Battery SOH Pipeline Notebook

This notebook reproduces the full pipeline: data loading, plotting, HI extraction, correlation-based feature selection, PSO-tuned SVR/LSTM/CNN training, and model saving.

In [ ]:
# Cell 1: Imports & global settings
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.svm import SVR
import pyswarms as ps

# Adjust this path if needed
DATA_DIR = "data"
BATTERIES = ["B0005", "B0006", "B0007"]

np.random.seed(42)


## Load Data

In [ ]:
# Cell 2: Load charge / discharge CSVs per battery
def load_battery(batt):
    path = os.path.join(DATA_DIR, batt)
    charge = pd.read_csv(os.path.join(path, f"{batt}_charge_data.csv"))
    discharge = pd.read_csv(os.path.join(path, f"{batt}_discharge_data.csv"))
    return charge, discharge

# Example: inspect one cycle from B0005
charge5, dis5 = load_battery("B0005")
print(charge5.columns, dis5.columns)

## Exploratory Plots

In [ ]:
# Cell 3: Plot example cycles for one battery
def plot_cycle(charge, discharge, cycle_id):
    c = charge[charge["cycle_id"] == cycle_id]
    d = discharge[discharge["cycle_id"] == cycle_id]
    fig, axs = plt.subplots(3, 2, figsize=(12, 8))
    axs = axs.ravel()
    axs[0].plot(c["time_s"], c["temp_C"]); axs[0].set_title("Charge Temp vs Time")
    axs[1].plot(d["time_s"], d["temp_C"]); axs[1].set_title("Discharge Temp vs Time")
    axs[2].plot(c["time_s"], c["voltage_V"]); axs[2].set_title("Charge Voltage")
    axs[3].plot(d["time_s"], d["voltage_V"]); axs[3].set_title("Discharge Voltage")
    axs[4].plot(c["time_s"], c["current_A"]); axs[4].set_title("Charge Current")
    axs[5].plot(d["time_s"], d["current_A"]); axs[5].set_title("Discharge Current")
    plt.tight_layout()
    plt.show()

plot_cycle(charge5, dis5, cycle_id=1)

## Health‐Indicator Extraction

In [ ]:
# Cell 4: HI extraction and feature matrix
def extract_HIs(cycle_charge, cycle_discharge):
    t_c, T_c, V_c, I_c = cycle_charge["time_s"].values, cycle_charge["temp_C"].values, cycle_charge["voltage_V"].values, cycle_charge["current_A"].values
    t_d, T_d, V_d, I_d = cycle_discharge["time_s"].values, cycle_discharge["temp_C"].values, cycle_discharge["voltage_V"].values, cycle_discharge["current_A"].values

    HI1 = t_c[np.argmax(T_c)]
    HI2 = T_c.max()
    HI3 = T_c.mean()
    HI4 = t_d[np.argmax(T_d)]
    HI5 = T_d.max()
    HI6 = T_d.mean()
    HI7 = V_c[0]
    V_at_ref = np.interp(1000, t_c, V_c)
    HI8 = 4.2 - V_at_ref
    idx_cut = np.argmax(V_c >= 4.2)
    HI9 = np.trapz(V_c[:idx_cut], t_c[:idx_cut])
    HI10 = V_d[0] - np.interp(100, t_d, V_d)
    HI11 = V_d.mean()
    idx_cv = idx_cut
    I_cv = I_c[idx_cv:]
    t_cv = t_c[idx_cv:]
    try:
        t12 = t_cv[np.where(I_cv <= 0.2)[0][0]] - t_cv[np.where(I_cv <= 0.5)[0][0]]
    except IndexError:
        t12 = np.nan
    HI12 = t12
    HI13 = I_c[0] - np.interp(100, t_cv, I_cv)
    HI14 = np.trapz(I_cv, t_cv)
    HI15 = I_d.mean()
    HI16 = np.trapz(V_c * I_c, t_c)

    return [HI1, HI2, HI3, HI4, HI5, HI6, HI7, HI8, HI9, HI10, HI11, HI12, HI13, HI14, HI15, HI16]

# Build the DataFrame
all_features = []
all_capacity = []
for batt in BATTERIES:
    ch, di = load_battery(batt)
    for cid in sorted(ch["cycle_id"].unique()):
        feat = extract_HIs(ch[ch["cycle_id"]==cid], di[di["cycle_id"]==cid])
        cap  = float(di[di["cycle_id"]==cid]["capacity_Ah"].iloc[0])
        all_features.append(feat)
        all_capacity.append(cap)

HI_cols = [f"HI{i}" for i in range(1,17)]
df = pd.DataFrame(all_features, columns=HI_cols)
df["capacity"] = all_capacity
df.head()

## Correlation & Feature Selection

In [ ]:
# Cell 5: correlation filtering (|r| >= 0.75)
corrs = df.corr()["capacity"].abs().loc[HI_cols]
to_keep = corrs[corrs >= 0.75].index.tolist()
print("Retained HIs:", to_keep)

X = df[to_keep].values
y = df["capacity"].values.reshape(-1,1)

## Train/Val/Test Split & Scaling

In [ ]:
# Cell 6: split & scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
X_train, X_val,  y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, shuffle=False)

scaler_X = MinMaxScaler().fit(X_train)
scaler_y = MinMaxScaler().fit(y_train)

X_train_s = scaler_X.transform(X_train)
X_val_s   = scaler_X.transform(X_val)
X_test_s  = scaler_X.transform(X_test)
y_train_s = scaler_y.transform(y_train).ravel()
y_val_s   = scaler_y.transform(y_val).ravel()
y_test_s  = scaler_y.transform(y_test).ravel()

## PSO for Hyperparameter Search (SVR)

In [ ]:
# Cell 7: PSO objective for SVR
def svr_obj(params):
    C, gamma, eps = params
    svr = SVR(kernel="rbf", C=C, gamma=gamma, epsilon=eps)
    svr.fit(X_train_s, y_train_s)
    preds = svr.predict(X_val_s)
    return np.sqrt(np.mean((preds - y_val_s)**2))

bounds = (np.array([1,1e-4,1e-4]), np.array([1000,1e-1,1e-1]))
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=3,
                                    options={"c1":0.5,"c2":0.3,"w":0.9},
                                    bounds=bounds)
best_cost, best_params = optimizer.optimize(lambda p: np.apply_along_axis(svr_obj,1,p), iters=30)
print("SVR PSO →", best_params)

## Train Final SVR

In [ ]:
# Cell 8: final SVR training & evaluate
C_opt, gamma_opt, eps_opt = best_params
model_svr = SVR(kernel="rbf", C=C_opt, gamma=gamma_opt, epsilon=eps_opt)
model_svr.fit(np.vstack([X_train_s, X_val_s]), np.concatenate([y_train_s, y_val_s]))

y_pred = model_svr.predict(X_test_s)
rmse = np.sqrt(np.mean((y_pred - y_test_s)**2))
print("Test RMSE (scaled):", rmse)

## PSO-LSTM & PSO-CNN (Outline)

In [ ]:
# Cell 9: reshape for sequence models
X_train_seq = X_train_s.reshape(-1, 1, X_train_s.shape[1])
X_val_seq   = X_val_s.reshape(-1, 1, X_val_s.shape[1])
X_test_seq  = X_test_s.reshape(-1, 1, X_test_s.shape[1])

def build_lstm(units, lr):
    m = Sequential([LSTM(int(units), input_shape=(1, X_train_s.shape[1])), Dense(1)])
    m.compile(loss="mse", optimizer="adam")
    return m

def lstm_obj(params):
    units, lr = params
    m = build_lstm(units, lr)
    m.fit(X_train_seq, y_train_s, validation_data=(X_val_seq, y_val_s),
          epochs=50, batch_size=32, verbose=0,
          callbacks=[EarlyStopping(patience=5)])
    pred = m.predict(X_val_seq).ravel()
    return np.sqrt(np.mean((pred - y_val_s)**2))

# Similar PSO for CNN can be added here.

## Save Models & Flask Snippet

In [ ]:
# Cell 10: save models and example Flask endpoint
import joblib
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

joblib.dump(model_svr, os.path.join(model_dir, "svr.pkl"))
# example for Keras models:
# model_lstm.save(os.path.join(model_dir, "lstm.h5"))
# model_cnn.save(os.path.join(model_dir, "cnn.h5"))

# Flask snippet (add to your existing backend):
# from flask import Flask, request, jsonify
# import joblib
# import numpy as np
# from tensorflow.keras.models import load_model
#
# app = Flask(__name__)
# svr  = joblib.load("models/svr.pkl")
#
# @app.route('/predict', methods=['POST'])
# def predict():
#     data = request.json
#     feats = np.array([data[h] for h in to_keep]).reshape(1,-1)
#     feats_s = scaler_X.transform(feats)
#     yhat = svr.predict(feats_s)
#     return jsonify({"soh_est": float(yhat)})